In [2]:
import os
import sys
import glob
import scipy
import pickle
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
from collections import defaultdict
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

%matplotlib inline
sns.set_style('whitegrid')
pd.set_option('display.max_rows', 100)
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['pdf.fonttype'] = 42
pd.set_option('display.max_columns', 100)

sys.path.append('/home/mattolm/Bio_scripts/')
import StatsTools

from IPython.display import display, HTML

from Bio import SeqIO

# Load FastANI data

In [3]:
FAdb = pd.read_pickle('/data1/bio_db/refseq/analysis/fastANI/mutlipleComps_v6.pickle')
FAdb = FAdb.set_index(['genome1', 'genome2'])

In [4]:
def _get_ani(FAdb, g1, g2):
    try:
        db = FAdb.loc[g1, g2]
    except:
        db = FAdb.loc[g2, g1]
    
    return db['fast_ani']

## Load MAG data with HGT

In [5]:
from tqdm import tqdm

def summarize_dnds_HGT(db, noFilt=False, fastANI=False):
    table = defaultdict(list)

    # Basic info
    table['reference'].append(db['reference'].tolist()[0])
    table['querry'].append(db['querry'].tolist()[0])

    # Number of comparisons
    table['total_comps'].append(len(db))
    table['failed_comps'].append(len(db[db['N_sites'] == 0]))
    table['successful_comps'].append(len(db[db['N_sites'] > 0]))
    table['considered_bases'].append(db[(db['N_sites'] > 0)]['al_len'].sum())
    table['percet_successful'].append((len(db[db['N_sites'] > 0]) / len(db)) * 100)
    
    # Filter to successful comparisons
    db = db[db['N_sites'] > 0]
    
    # dN/dS stuff
    N = db['N_sites'].sum()
    table['N_sites'].append(N)
    dN = db['N_changed'].sum()
    table['N_changed'].append(dN)
    S = db['S_sites'].sum()
    table['S_sites'].append(S)
    dS = db['S_changed'].sum()
    table['S_changed'].append(dS)
    if (N > 0) & (S > 0) & (dS > 0):
        table['dN/dS'].append(((dN/N) / (dS/S)))
    else:
        table['dN/dS'].append(np.nan)

    # Calculate ANI if you must
    if fastANI == False:
        fastANI = sum([p * l for p, l in zip(db['p_inden'], db['al_len'])]) \
                    / db['al_len'].sum()
    table['fast_ani'].append(fastANI)
        
    # Filter to only ones with a full alignment
    if noFilt:
        hdb = db[(db['al_len'] > 500)]
    else:
        hdb = db[(db['al_len'] > 500) & (db['al_len'] < 1000)]
                 
    ex_iden = _calc_expected(hdb, fastANI)
    table['counted_comps'].append(len(hdb))
    table['identical_comps'].append(len(hdb[hdb['p_inden'] > 99.99]))
    table['expected_identicle'].append(ex_iden)
    
    try:
        table['percent_enriched'].append(((len(hdb[hdb['p_inden'] > 99.99]) - int(ex_iden)) / len(hdb))*100)
        hani = sum([p * l for p, l in zip(hdb['p_inden'], hdb['al_len'])]) \
                / hdb['al_len'].sum()
        table['filtered_ani'].append(hani)
        table['filtered_af'].append(db['al_len'].sum() / db['qry_len'].sum())
    except:
        table['percent_enriched'].append(0)
        if not noFilt:
            table['filtered_ani'].append(0)
            table['filtered_af'].append(0)   

    Ndb = pd.DataFrame(table)
    return Ndb

def _calc_expected(db, ani):
    expected = 0
    prob_diff = (100-ani) / 100
    prob_same = ani / 100
    for i, row in db.iterrows():
        # probability of this row being 0
        #p = prob_diff * int(row['al_len'])
        p = prob_same ** int(row['al_len'])
        if p > 1:
            expected += 1
        else:
            expected += p
    return expected

def parse_to_genomeLevel_HGT(Ddb, noFilt=False):
    Tdbs = []
    for query, qdb in tqdm(Ddb.groupby('querry')):
        for ref, db in qdb.groupby('reference'):
            # get the FastANI
            try:
                fastANI = _get_ani(FAdb, db['reference'].tolist()[0], db['querry'].tolist()[0])
            except:
                fastANI = 0
            
            tdb = summarize_dnds_HGT(db, noFilt=noFilt, fastANI=fastANI)
            Tdbs.append(tdb)
    return pd.concat(Tdbs)

In [6]:
def load_dnds(wd_loc):
    print(wd_loc)
    
    #baseloc = wd_loc + 'data/dNdS_data/genomeWide_dNdS_info.csv'
    baseloc = wd_loc + 'data/dNdS_data/detailed_dNdS_info.csv'
    if os.path.exists(baseloc):
        Tdb = pd.read_csv(baseloc)
    else:
        print("ERROR")
        #Tdb = pd.concat([pd.read_csv(t) for t in glob.glob(wd_loc + 'data/dNdS_data/detailed*_chunk_*.csv')])
        
    return parse_to_genomeLevel_HGT(Tdb, noFilt=True)

# Tdb = pd.DataFrame()
# for floc in ['/data1/bio_db/refseq/analysis/MAGlists_2/goANI_oceanList/',
#              '/data1/bio_db/refseq/analysis/MAGlists_2/goANI_soilList/',
#              '/data1/bio_db/refseq/analysis/MAGlists_2/goANI_refseqTheta/',
#              '/data1/bio_db/refseq/analysis/MAGlists_2/goANI_infantList//']:
#     tdb = load_dnds(floc)
#     tdb['method'] = floc.split('/')[6].split('_')[1]
#     Tdb = Tdb.append(tdb)

In [7]:
# Part 1 has ocean and soil!
#Tdb.to_pickle('/data1/bio_db/refseq/analysis/dn_ds/data_tables/genomeWide_dNdS_info_v6_pt1.pickle')

In [8]:
Tdb = pd.DataFrame()
for floc in ['/data1/bio_db/refseq/analysis/MAGlists_2/goANI_infantList/']:
    tdb = load_dnds(floc)
    tdb['method'] = floc.split('/')[6].split('_')[1]
    Tdb = Tdb.append(tdb)

/data1/bio_db/refseq/analysis/MAGlists_2/goANI_infantList/


100%|██████████| 1952/1952 [12:22:00<00:00,  1.22it/s]   


In [9]:
# Part 2 has infant!
#Tdb.to_pickle('/data1/bio_db/refseq/analysis/dn_ds/data_tables/genomeWide_dNdS_info_v6_pt2.pickle')

# Part 3 has RefSeq

In [ ]:
Tdb = pd.DataFrame()
for floc in ['/data1/bio_db/refseq/analysis/MAGlists_2/goANI_refseqTheta/']:
    tdb = load_dnds(floc)
    tdb['method'] = floc.split('/')[6].split('_')[1]
    Tdb = Tdb.append(tdb)

/data1/bio_db/refseq/analysis/MAGlists_2/goANI_refseqTheta/


 40%|████      | 1764/4363 [9:31:04<9:02:46, 12.53s/it]   

In [ ]:
#Tdb.to_pickle('/data1/bio_db/refseq/analysis/dn_ds/data_tables/genomeWide_dNdS_info_v6_pt3.pickle')

In [9]:
print()